https://pytorch.org/blog/finetune-llms/

https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing

# 1. Load Environment & Huggingface, W&B login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_03"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Dataset Load

- 이 시스템에서는 작업을 외장하드에서 하고 있기 때문에 캐쉬폴더를 별도로 지정(In this system, tasks are being performed on an external hard drive, so a separate cache folder is specified.)

In [3]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [4]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [5]:
ko_instruction_01[100]

{'id': 'alpaca_{idx}',
 'instruction': '다음 문장을 3인칭으로 다시 작성합니다.',
 'input': '나는 불안하다',
 'output': '불안해합니다.'}

In [6]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [7]:
# ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.9)

# 3. Model Load

- QLoRA로 finetune을 하기위해 일반 7B 모델을 BitsAndBytes를 통해 양자화(4bit quantization)하여 로드, 훈련속도 향상을 위해 데이터 타입을 fp32가 아니라 fp16으로 로드(차후에는 bf16으로 로드해서 훈련예정)
- To finetune with QLoRA, a standard 7B model is quantized (4-bit quantization) through BitsAndBytes for loading, and to improve training speed, the data type is loaded as fp16 instead of fp32 (with plans to load and train with bf16 in the future).


```python

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
```

### *Reference

- https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py
- https://colab.research.google.com/drive/19AFEOrCI6-bc7h9RTso_NndRwXJRaJ25?usp=sharing

In [8]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [9]:
checkpoint_dir = "./checkpoint/experi_04"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}

In [10]:
# 4bit QLoRA 학습을 위한 설정
bnb_4bit_compute_dtype = "bfloat16" # 코랩 무료버전에서 실행 시 "float16"를 사용하세요
bnb_4bit_quant_type = "nf4"
use_4bit = True
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [11]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device_map, cache_dir="/mnt/t7/.cache/huggingface/models")

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

### 1) PEFT훈련을 위한 CONFIG설정, 이 부분은 Trainable Parameter를 확인하지 않아도 QLoRA를 위해서는 무조건 해야 한다.("For PEFT training configuration, this step must be done unconditionally for QLoRA, even without checking for Trainable Parameters.")

In [12]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    inference_mode=False,
    task_type="CAUSAL_LM",
)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

tokenizer.pad_token = tokenizer.eos_token

# # 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
tokenizer.padding_side = "right"

### 4) Prompt 설정(Prompt configuration)

- Prompt는 아래와 같음(The prompt is as follows.)
> \#\#\# System: content  
> \#\#\# Human: content  
> \#\#\# Assistant: content  

- Prompt에 맞춰서 Dataset을 재가공(Reprocessing the Dataset according to the Prompt)

In [14]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [15]:
train_dataset = ko_instruction_01.map(template_dataset, remove_columns=list(ko_instruction_01.features), num_proc=10)

In [16]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 152630
})

In [17]:
train_dataset["text"][1000]

'### instruction: 인공 신경망의 두 가지 주요 구성 요소는 무엇인가요?\n\n### output: 인공 신경망의 두 가지 주요 구성 요소는 뉴런(또는 노드)과 뉴런 간의 상호 연결(또는 시냅스)입니다. 뉴런은 계층으로 구성되어 있으며, 입력 계층은 데이터를 받아들이고, 출력 계층은 결과를 생성하며, 그 사이에 숨겨진 계층은 연산을 수행합니다. 뉴런 간의 상호 연결을 통해 정보의 흐름이 이루어지며, 학습 과정에서 뉴런의 강점 또는 가중치가 조정되어 네트워크의 성능이 향상됩니다.</s>'

In [18]:
train_dataset["text"][3000]

"### instruction: 이 코드 스니펫을 Keras 대신 PyTorch로 다시 생성할 수 있나요?\n\n### output: 물론 다음은 파이토치에서 구현된 신경망을 사용하여 지도 학습 접근법을 사용하여 이진 행렬을 완성하는 방법의 예입니다:'''pythonimport numpy as npsklearn.model_selection에서 train_test_split을 가져옵니다.sklearn.metrics에서 정확도_점수 가져오기토치 가져오기torch.nn을 nn으로 가져옵니다.torch.optim을 optim으로 가져옵니다.# 신경망을 사용하여 누락된 값을 대입하는 함수를 만듭니다.def complete_matrix(matrix):    # 행렬을 관측값과 결측값으로 분할합니다.    observed_values = matrix[matrix != 0]    missing_mask = matrix == 0    missing_values = matrix[missing_mask]    # 데이터를 훈련 및 테스트 집합으로 분할    X_train, X_test, y_train, y_test = train_test_split(observed_values, missing_values, test_size=0.2)    # 신경망 정의 및 훈련    model = nn.Sequential(        nn.Linear(1, 32),        nn.ReLU(),        nn.Linear(32, 1),        nn.Sigmoid()    )    criterion = nn.BCELoss()    optimizer = optim.Adam(model.parameters())    X_train = torch.from_numpy(X_train).float()    y_train = torch.from_numpy(y_train).float()    X_test = torch.from_numpy(X_test).float()    for epoch in ran

In [19]:
train_dataset["text"][3230]

'### instruction: 30분 이내에 준비할 수 있는 식사를 제안하세요.\n\n### output: 30분 이내에 준비할 수 있는 한 끼 식사는 닭고기 볶음입니다. 간단한 레시피를 소개합니다:\n\n재료\n- 얇게 썬 뼈 없는 껍질 없는 닭 가슴살 1파운드\n- 식물성 기름 2큰술\n- 다진 마늘 2쪽\n- 다진 생강 1큰술\n- 얇게 썬 붉은 피망 1개\n- 브로콜리 꽃 1 컵\n- 슈가 스냅 완두콩 1 컵\n- 얇게 썬 당근 1컵\n- 간장 1/4 컵\n- 닭 육수 1/4 컵\n- 옥수수 전분 1 큰술\n- 꿀 2작은술\n- 소금과 후추, 맛보기\n\n만드는 방법\n1. 큰 팬에 기름을 두르고 중간보다 센 불에 올립니다. 마늘과 생강을 넣고 30초간 조리합니다.\n2. 닭고기 스트립을 넣고 갈색이 나고 완전히 익을 때까지 약 5분간 조리합니다. 팬에서 닭고기를 꺼내 따로 보관합니다.\n3. 3. 같은 팬에 붉은 피망, 브로콜리, 슈가 스냅 완두콩, 얇게 썬 당근을 추가합니다. 야채가 부드러워질 때까지 6~10분간 저어가며 조리합니다.\n4. 작은 볼에 간장, 닭 육수, 옥수수 전분, 꿀을 넣고 휘젓습니다. 익힌 야채 위에 소스를 붓고 소스가 걸쭉해질 때까지 약 2분간 저어줍니다.\n5. 익힌 닭고기를 팬에 다시 넣고 모든 것이 소스에 잘 코팅 될 때까지 저어줍니다. 추가로 2-3 분 더 조리합니다.\n6. 소금과 후추로 간을하여 맛을 낸다. 밥이나 국수와 함께 제공합니다.\n\n그게 다입니다! 30분 이내에 준비할 수 있는 건강하고 맛있는 식사입니다. 맛있게 드세요!</s>'

In [20]:
train_dataset["text"][12312]

"### instruction: 직원의 이름과 급여를 저장할 수 있는 데이터 구조의 예를 만듭니다.\n\n### output: 직원의 이름과 급여를 저장할 수 있는 데이터 구조의 한 예로, 키는 직원의 이름이고 값은 급여인 사전을 들 수 있습니다.\n\n```\nemployee_salaries = {\n    'John Doe': 50000,\n    'Jane Smith': 60000,\n    'Bob Johnson': 55000,\n    'Emily Davis': 65000\n}\n```\n\n이 예제에서 `employee_salaries` 사전은 직원 4명의 이름과 급여를 저장합니다. 키는 직원의 이름(예: 'John Doe')이고 값은 급여(예: 50000)입니다. 따라서 이름을 참조하기만 하면 모든 직원의 급여 정보에 쉽게 액세스할 수 있습니다.</s>"

In [21]:
# test_dataset = ko_instruction_01['test'].map(template_dataset, remove_columns=list(ko_instruction_01['test'].features), num_proc=10)

# 4. Configure Train arguments & Define Trainer

In [22]:
training_arguments = TrainingArguments(
    output_dir=checkpoint_dir,
    save_steps=200,
    save_total_limit=3, # 가장 최근 체크포인트 3개만 저장합니다.
    logging_steps=30,
    report_to="wandb",
    learning_rate=1e-5,
    max_grad_norm=0.3,
    num_train_epochs=2, # epochs 대신 max_steps을 기준으로 할 수 있습니다.
    warmup_ratio=0.02,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit", # paged_adamw_8bit 사용시 메모리를 더 절약할 수 있지만 loss가 0으로 떨어지는 문제가 있습니다.
    group_by_length=True,
    fp16 = False, # 코랩 무료버전에서 실행 시 "True"를 사용하세요
    bf16 = True, # 코랩 무료버전에서 실행 시 "False"를 사용하세요
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=1024,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=False,
# )

In [24]:
trainer.train()


  0%|          | 0/305260 [00:00<?, ?it/s]

{'loss': 2.467, 'grad_norm': 0.1328125, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 3.2204, 'grad_norm': 0.11279296875, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.6088, 'grad_norm': 0.2470703125, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.9207, 'grad_norm': 0.16015625, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 3.2477, 'grad_norm': 2.34375, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.6076, 'grad_norm': 0.341796875, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 3.2067, 'grad_norm': 0.1748046875, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.6031, 'grad_norm': 0.73046875, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.8153, 'grad_norm': 0.6328125, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.7713, 'grad_norm': 1.6796875, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.2137, 'grad_norm': 0.3359375, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.7726, 'grad_norm': 0.2734375, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 2.3876, 'grad_norm': 0.87890625,

KeyboardInterrupt: 

### 허깅페이스에서 훈련 도중 러닝레이트가 수정 안되는 경우가 많다 한다. 
- 스케쥴러 변경을 시도해 보기로 한다. 

In [ ]:
output_dir = "./results/experi_04"

In [ ]:
trainer.model.save_pretrained(output_dir)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3c877566-c0c7-4b66-a1ec-258126076aba)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(
